In [1]:
from recbole.quick_start import run_recbole
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_logger, init_seed
from logging import getLogger
from recbole.model.sequential_recommender import NextItNet
from recbole.trainer import Trainer

c:\Users\yubra\anaconda3\envs\rec\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
parameter_dict = {
    # environment config : Sequential Recommendation
'data_path': "data1/",
'checkpoint_dir': "saved",
'reproducibility':' False',
'show_progress': 'True',
# dataset config : Sequential Recommendation
'USER_ID_FIELD': 'user_id',
'ITEM_ID_FIELD': 'item_id',
'TIME_FIELD': 'timestamp',
'load_col':
    {'inter': ['user_id', 'item_id','timestamp']},
'ITEM_LIST_LENGTH_FIELD': 'item_length',
'LIST_SUFFIX': '_list',
'MAX_ITEM_LIST_LENGTH': 50,
'train_neg_sample_args': 'None',

# training and evaluation config : Sequential Recommendation
'epochs': 2,
'train_batch_size': 2048,
'eval_batch_size': 2048,
'learner': 'adam',
'eval_args': {
    'group_by': 'user',
    'order': 'TO',
    'split': {'LS': 'valid_and_test'},
    'mode': 'full',
},
'metrics': ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision'],
'topk': 10,
'valid_metric': 'MRR@10',
'metric_decimal_place': 4
}

In [5]:
config = Config(model='NextItNet',
                    dataset='dataset',
                    config_dict=parameter_dict)



init_seed(config['seed'], config['reproducibility'])


init_logger(config)
logger = getLogger()


logger.info(config)

13 Jul 14:03    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020


state = INFO
reproducibility = False
data_path = data1/dataset
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 2
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'group_by': 'user', 'order': 'TO', 'split': {'LS': 'valid_and_test'}, 'mode': 'full'}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 2048
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME_

In [12]:
dataset = create_dataset(config)
logger.info(dataset)

13 Jul 14:26    INFO  dataset
The number of users: 85871
Average actions of users: 3.7697100267846744
The number of items: 2966
Average actions of items: 109.17537942664418
The number of inters: 323705
The sparsity of the dataset: 99.87290404156785%
Remain Fields: ['user_id', 'item_id', 'timestamp']


In [13]:
train_data, valid_data, test_data = data_preparation(config, dataset)

13 Jul 14:26    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
13 Jul 14:26    INFO  [Evaluation]: eval_batch_size = [2048] eval_args: [{'group_by': 'user', 'order': 'TO', 'split': {'LS': 'valid_and_test'}, 'mode': 'full'}]


In [14]:
model = NextItNet(config, train_data.dataset)
logger.info(model)

13 Jul 14:27    INFO  NextItNet(
  (item_embedding): Embedding(2966, 64, padding_idx=0)
  (residual_blocks): Sequential(
    (0): ResidualBlock_b(
      (conv1): Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1))
      (ln1): LayerNorm((64,), eps=1e-08, elementwise_affine=True)
      (conv2): Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), dilation=(2, 2))
      (ln2): LayerNorm((64,), eps=1e-08, elementwise_affine=True)
    )
    (1): ResidualBlock_b(
      (conv1): Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), dilation=(4, 4))
      (ln1): LayerNorm((64,), eps=1e-08, elementwise_affine=True)
      (conv2): Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), dilation=(8, 8))
      (ln2): LayerNorm((64,), eps=1e-08, elementwise_affine=True)
    )
    (2): ResidualBlock_b(
      (conv1): Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1))
      (ln1): LayerNorm((64,), eps=1e-08, elementwise_affine=True)
      (conv2): Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), dilation=(2, 2))
 

In [15]:
trainer = Trainer(config, model)
best_valid_score, best_valid_result = trainer.fit(train_data)

13 Jul 14:33    INFO  epoch 0 training [time: 333.84s, train loss: 404.7268]
13 Jul 14:33    INFO  Saving current: saved\NextItNet-Jul-13-2023_14-27-49.pth
13 Jul 14:39    INFO  epoch 1 training [time: 355.32s, train loss: 399.3483]
13 Jul 14:39    INFO  Saving current: saved\NextItNet-Jul-13-2023_14-27-49.pth
